## Predicting RBP Binding with CNN and Transformer

### Imports

In [1]:
import os 
import torch
import torch.nn as nn 
import torch.utils.data
from torch.nn.modules.pooling import MaxPool2d

import numpy as np 
import math
import pickle
import pandas as pd
from pathlib import Path
from google.colab import drive
import random

assert(torch.cuda.is_available())
device = "cuda" 

drive.mount('/content/drive') 

Mounted at /content/drive


### Datasets

##### ENCODE

In [2]:
root_dir = '/content/drive/MyDrive/ML4FG/Project' 

In [3]:
encode_dir = Path(root_dir + '/datasets/ENCODE')

encode_df = pd.read_csv(encode_dir / 'encode_eclip_idr.bed.gz', sep = '\t')
encode_df = encode_df.sort_values(['chr', 'start']).drop_duplicates()
encode_df 

,chr,start,end,strand,fold,nlog10p,cell,target
565739,chr1,15922,15985,-,4.336078,6.512251,K562,AQR
494169,chr1,16230,16331,-,3.344423,6.274660,K562,BUD13
562861,chr1,16250,16319,-,4.045827,7.122894,K562,AQR
558839,chr1,16462,16510,-,5.292347,6.083708,K562,AQR
278941,chr1,17451,17573,-,3.361457,4.291127,K562,CSTF2T
...,...,...,...,...,...,...,...,...
438950,chrY,19744401,19744473,-,3.464966,8.168405,HepG2,BCLAF1
191326,chrY,19744489,19744548,-,3.048818,4.066013,HepG2,PPIG
671368,chrY,20579772,20579892,+,4.660723,5.507490,HepG2,U2AF2
159170,chrY,20580800,20580864,+,4.144179,3.393384,HepG2,QKI


In [4]:
target_set = set(encode_df['target'])
target_to_idx = dict(zip(target_set, list(range(len(target_set)))))
idx_to_target = dict(zip(list(range(len(target_set))), target_set)) 

##### rnbs_HNRNPK

In [5]:
rnbs_HNRNPK_dir = Path(root_dir + '/datasets/rnbs_HNRNPK')

HNRNPK_0 = pd.read_csv(rnbs_HNRNPK_dir / 'HNRNPK_0.fasta.gz', header=None, names = ['Sequence'], sep = '\t')
HNRNPK_0 

,Sequence
0,AGGACCTGACCATACGATGA
1,ACTAAATCTCATGCAGGATA
2,TTATAGCCCGATTAGGAGGG
3,ATCGCCAATTGTCTTCAGAT
4,CATTATCGCTTTATACAACT
...,...
11509893,ACAGTCTGGTTTCGAACGCG
11509894,ACGAAATCTAGAGTAACTTA
11509895,GCCATGAGATCATACGTCTT
11509896,ACTCCACCGTTAGTATCCTT


##### hg38

In [6]:
genome = pickle.load(open('/content/drive/MyDrive/ML4FG/Project/hg38.pkl',"rb")) 

### Functions


In [7]:
def train_val_test_split(df):
  chr_set = set(df['chr'])
  valtest_chr_set = set(['chr1', 'chr2', 'chr3'])
  train_chr_set = chr_set - valtest_chr_set

  train_df = df.loc[df['chr'].isin(train_chr_set)]
  val_df = df.loc[df['chr'].isin(['chr2', 'chr3'])]
  test_df = df.loc[df['chr'] == 'chr1']

  return train_df, val_df, test_df


In [8]:
encode_train, encode_val, encode_test = train_val_test_split(encode_df) 

In [9]:
def get_longest_seq_len(df):
  start_lst = list(df['start'])
  end_lst = list(df['end'])
  seq_len = [end - start for start, end in zip(start_lst, end_lst)] 
  max_seq_len = list(set(seq_len))[-1]

  return max_seq_len


In [10]:
max_seq_len = get_longest_seq_len(encode_df) 

In [11]:
def one_hot(seq):
  d = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

  one_hot_enc = np.zeros((4, len(seq)))

  for i in range(len(seq)):
    el = seq[i]
    if el != 'N':
      one_hot_enc[d[el], i] = 1.0

  return one_hot_enc 


In [12]:
def pad_enc(one_hot_enc, max_seq_len):
  padded_enc = np.zeros((4, max_seq_len))
  padded_enc[:, 0:one_hot_enc.shape[1]] = one_hot_enc
  
  return padded_enc 


In [13]:
# convert negative strand to its positive equivalence
def strand_normalize(seq):
  d = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N':'N'}

  modified_seq =''

  for el in seq:
    modified_seq += d[el]

  return modified_seq 
  

In [14]:
def collect_positive_samples(df):
  start_lst = list(df['start'])
  end_lst = list(df['end'])
  strand_lst = list(df['strand'])

  positive_samples = list()

  for i in range(df.shape[0]):
    current_start, current_end = start_lst[i], end_lst[i]

    if i == 0:
      positive_samples.append([(start_lst[i], end_lst[i]), strand_lst[i]])

    else:
      previous_start, previous_end = positive_samples[-1][0]

      if previous_end < current_start:
        positive_samples.append([(current_start, current_end), strand_lst[i]])

      else:
        positions = [previous_start, previous_end, current_start, current_end]
        positions.sort()

        positive_samples[-1] = [(positions[0], positions[-1]), strand_lst[i-1]]

  return positive_samples 
  

In [15]:
train_positive_samples = collect_positive_samples(encode_train)  

In [16]:
def get_positive_label(target, target_to_idx):
  label = np.zeros(len(target_to_idx))
  label[target_to_idx[target]] = 1

  return label 
  

In [17]:
def mtx_to_idx_label(mtx):
  indices = torch.zeros(mtx.shape[0], dtype=torch.int)
  for i in range(mtx.shape[0]):
    row = mtx[i,:]
    indices[i] = list(row).index(1)

  return indices 
  

### Dataloader

In [18]:
class BedPeaksDataset(torch.utils.data.IterableDataset):

    def __init__(self, eclip_data, genome, context_length, target_to_idx):
        super(BedPeaksDataset, self).__init__()
        self.eclip_data = eclip_data
        self.genome = genome
        self.context_length = context_length
        self.target_to_idx = target_to_idx 

    def __iter__(self): 
        for i,row in enumerate(self.eclip_data.itertuples()):
            midpoint = int(.5 * (row.start + row.end))
            seq = self.genome[row.chr][midpoint - self.context_length//2:midpoint + self.context_length//2]
            if row.strand == '-':
              seq = strand_normalize(seq)
            yield(one_hot(seq), get_positive_label(row.target, self.target_to_idx)) # positive example 


    def __len__(self):
        return self.eclip_data.shape[0]


In [19]:
context_length = 100

encode_train_dataset = BedPeaksDataset(encode_train, genome, context_length, target_to_idx)
encode_train_dataloader = torch.utils.data.DataLoader(encode_train_dataset, batch_size=1028, num_workers = 0) 

encode_val_dataset = BedPeaksDataset(encode_val, genome, context_length, target_to_idx)
encode_val_dataloader = torch.utils.data.DataLoader(encode_val_dataset, batch_size=1028)

encode_test_dataset = BedPeaksDataset(encode_test, genome, context_length, target_to_idx)
encodetest_dataloader = torch.utils.data.DataLoader(encode_test_dataset, batch_size=1028) 

### Convolutional Neural Network

In [42]:
class ConvBasicNet(nn.Module): 

    def __init__(self, padding=False):
        super().__init__()

        self.padding = padding
        self.conv_model = self.get_conv_layers()
        self.final_max_pool = self.final_pool_layer()
        self.fc_model = self.get_fc_layers()

    def get_conv_layers(self):
        
        layers = nn.Sequential(
            nn.Conv1d(4, 32, 5),
            nn.BatchNorm1d(32),
            nn.MaxPool1d(2, 2),
            nn.ReLU(),
            nn.Conv1d(32, 16, 5),
            nn.BatchNorm1d(16),
            nn.MaxPool1d(2, 2),
            nn.ReLU()
        )

        return layers

    def final_pool_layer(self):
        
        layer = nn.MaxPool1d(2, 2) 
        
        return layer

    def get_fc_layers(self):
        
        if self.padding==False:
          layers = nn.Sequential(
              nn.Linear(176, 512),
              nn.ReLU(),
              nn.Linear(512, 1024),
              nn.ReLU(),
              nn.Linear(1024, 150)
          )

        elif self.padding:
          layers = nn.Sequential(
              nn.Linear(912, 1024),
              nn.ReLU(),
              nn.Linear(1024, 2048),
              nn.ReLU(),
              nn.Linear(2048, 150)
          )

        return layers

    def register_grad_hook(self, grad):
        self.grad = grad 


    def forward(self, x):

        x = self.conv_model(x) 
        h = x.register_hook(self.register_grad_hook)

        x = self.final_max_pool(x)
        x = nn.Flatten()(x) 
        x = self.fc_model(x)

        return x    

    def get_gradient_activations(self):
        return self.grad

    def get_final_conv_layer(self, x):
        return self.conv_model(x) 


In [23]:
class Classifier():

    def __init__(self, name, model, dataloaders, class_names, use_cuda=False):
        
        '''
        @name: Experiment name. Will define stored results etc. 
        @model: GradBasicNet()
        @dataloaders: Dictionary with keys train, val and test and corresponding dataloaders
        @class_names: list of classes, where the idx of class name corresponds to the label used for it in the data
        @use_cuda: whether or not to use cuda
        '''
        
        self.name = name
        if use_cuda and not torch.cuda.is_available():
            raise Exception("Asked for CUDA but GPU not found")
            
        self.use_cuda = use_cuda
        self.model = model.to('cuda' if use_cuda else 'cpu')

        self.criterion = nn.CrossEntropyLoss()
        self.optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

        self.dataloaders = dataloaders 
        self.class_names = class_names
        self.activations_path = os.path.join('activations', self.name)
        self.kernel_path = os.path.join('kernel_viz', self.name)

        save_path = os.path.join(os.getcwd(), 'models', self.name)
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        if not os.path.exists(self.activations_path):
            os.makedirs(self.activations_path)

        if not os.path.exists(self.kernel_path):
            os.makedirs(self.kernel_path)
            
        self.save_path = save_path

    def train(self, epochs, save=True):
        '''
        @epochs: number of epochs to train
        @save: whether or not to save the checkpoints
        '''

        best_val_accuracy = - math.inf

        for epoch in range(epochs):

            self.model.train()

            batches_in_pass = len(self.dataloaders['train'])
            
            loss_total = 0.0 # Record the total loss within a few steps
            epoch_loss = 0.0 # Record the total loss for each epoch
            
            for idx, data in enumerate(self.dataloaders['train']):
               
              inputs, labels = data
            
              inputs = inputs.to('cuda', dtype=torch.float if self.use_cuda else 'cpu') 
              labels = labels.to('cuda', dtype=torch.float if self.use_cuda else 'cpu')

              outputs = self.model(inputs)
              train_loss = self.criterion(outputs, labels)

              if idx != 0 and idx % 10000 == 0: 
                print(loss_total.item())
                loss_total = 0

              loss_total += train_loss
              epoch_loss += train_loss 

              self.optim.zero_grad()
              train_loss.backward() 
              self.optim.step()

            '''Give validation'''
            epoch_loss /= batches_in_pass

            self.model.eval()
            
            #DO NOT modify this part
            correct = 0.0
            total = 0.0
            for idx, data in enumerate(self.dataloaders['val']):

                inputs, labels = data
                
                inputs = inputs.to('cuda', dtype=torch.float if self.use_cuda else 'cpu')
                labels = labels.to('cuda', dtype=torch.float if self.use_cuda else 'cpu')

                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)

                total += labels.shape[0]
                indices_labels = (mtx_to_idx_label(labels)).to('cuda')
                correct += (predicted == indices_labels).sum().item()


            epoch_accuracy = 100 * correct / total

            print(f'Train Epoch Loss (Avg): {epoch_loss}')
            print(f'Validation Epoch Accuracy:{epoch_accuracy}')
            
            if save:
                
                torch.save(self.model.state_dict(), os.path.join(self.save_path, f'epoch_{epoch}.pt'))
                
                if epoch_accuracy > best_val_accuracy:

                    torch.save(self.model.state_dict(), os.path.join(self.save_path, 'best.pt'))
                    print('save model', os.path.join(self.save_path, f'epoch_{epoch}.pt'))
                    best_val_accuracy = epoch_accuracy

        print('Finish training!')                       

    
    def evaluate(self):
        
        try:
            assert os.path.exists(os.path.join(self.save_path, 'best.pt'))
            
        except:
            print('It appears you are testing the model without training. Please train first')
            return
        
        self.model.load_state_dict(torch.load(os.path.join(self.save_path, 'best.pt')))
        self.model.eval()

        #total = len(self.dataloaders['test'])
        
        correct = 0.0
        total = 0.0
        for idx, data in enumerate(self.dataloaders['test']):
            
            inputs, labels = data
            inputs = inputs.to('cuda', dtype=torch.float if self.use_cuda else 'cpu')
            labels = labels.to('cuda', dtype=torch.float if self.use_cuda else 'cpu')

            outputs = self.model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.shape[0]
            indices_labels = (mtx_to_idx_label(labels)).to('cuda')
            correct += (predicted == indices_labels).sum().item()
                
        print(f'Accuracy: {100 * correct/total}%')
        

In [80]:
experiment_name = 'basic' 
model_name = 'basic' 

dataloaders = {'train': encode_train_dataloader, 'val' : encode_val_dataloader, 'test': encode_train_dataloader, 'mapping': target_set}
model = ConvBasicNet()
classifier = Classifier(experiment_name, model, dataloaders, target_set, use_cuda=True) 

In [ ]:
classifier.train(epochs=20)
classifier.evaluate() 

Train Epoch Loss (Avg): 4.409198760986328
Validation Epoch Accuracy:11.277369885448945
save model /content/models/basic/epoch_0.pt
Train Epoch Loss (Avg): 4.119940280914307
Validation Epoch Accuracy:14.886984357406735
save model /content/models/basic/epoch_1.pt
Train Epoch Loss (Avg): 3.925590991973877
Validation Epoch Accuracy:15.824051068739939
save model /content/models/basic/epoch_2.pt
Train Epoch Loss (Avg): 3.8540444374084473
Validation Epoch Accuracy:16.287466383778604
save model /content/models/basic/epoch_3.pt
Train Epoch Loss (Avg): 3.812438726425171
Validation Epoch Accuracy:16.72482621925686
save model /content/models/basic/epoch_4.pt
Train Epoch Loss (Avg): 3.778705358505249
Validation Epoch Accuracy:17.098908461516988
save model /content/models/basic/epoch_5.pt
Train Epoch Loss (Avg): 3.7481296062469482
Validation Epoch Accuracy:17.514865581642052
save model /content/models/basic/epoch_6.pt
Train Epoch Loss (Avg): 3.7195050716400146
Validation Epoch Accuracy:17.8982533523

In [31]:
!git clone https://github.com/xypan1232/iDeepS 

Cloning into 'iDeepS'...
remote: Enumerating objects: 9858, done.
remote: Total 9858 (delta 0), reused 0 (delta 0), pack-reused 9858
Receiving objects: 100% (9858/9858), 547.92 MiB | 12.15 MiB/s, done.
Resolving deltas: 100% (3603/3603), done.
Checking out files: 100% (3530/3530), done.


### CNN with various input lengths (padded)

In [24]:
class VariedBedPeaksDataset(torch.utils.data.IterableDataset):

    def __init__(self, eclip_data, genome, max_seq_len, target_to_idx):
        super(VariedBedPeaksDataset, self).__init__()
        self.eclip_data = eclip_data
        self.genome = genome
        self.max_seq_len = max_seq_len
        self.target_to_idx = target_to_idx 

    def __iter__(self): 
        for i,row in enumerate(self.eclip_data.itertuples()):
            midpoint = int(.5 * (row.start + row.end))
            seq = self.genome[row.chr][row.start:row.end]
            if row.strand == '-':
              seq = strand_normalize(seq) 
            yield(pad_enc(one_hot(seq), self.max_seq_len), get_positive_label(row.target, self.target_to_idx)) # positive example 


    def __len__(self):
        return self.eclip_data.shape[0]


In [25]:
encode_train_dataset = VariedBedPeaksDataset(encode_train, genome, max_seq_len, target_to_idx)
encode_train_dataloader = torch.utils.data.DataLoader(encode_train_dataset, batch_size=1028, num_workers = 0) 

encode_val_dataset = VariedBedPeaksDataset(encode_val, genome, max_seq_len, target_to_idx)
encode_val_dataloader = torch.utils.data.DataLoader(encode_val_dataset, batch_size=1028)

encode_test_dataset = VariedBedPeaksDataset(encode_test, genome, max_seq_len, target_to_idx)
encodetest_dataloader = torch.utils.data.DataLoader(encode_test_dataset, batch_size=1028) 

In [43]:
experiment_name = 'basic_pad'  #Provide name to model experiment
model_name = 'basic_pad' 

dataloaders = {'train': encode_train_dataloader, 'val' : encode_val_dataloader, 'test': encode_train_dataloader, 'mapping': target_set}
model = ConvBasicNet(padding=True)
classifier = Classifier(experiment_name, model, dataloaders, target_set, use_cuda=True) 

In [44]:
classifier.train(epochs=20)
classifier.evaluate() 

Train Epoch Loss (Avg): 4.349259853363037
Validation Epoch Accuracy:12.032978792700744
save model /content/models/basic_pad/epoch_0.pt
Train Epoch Loss (Avg): 3.9622132778167725
Validation Epoch Accuracy:15.969217312005062
save model /content/models/basic_pad/epoch_1.pt
Train Epoch Loss (Avg): 3.811177968978882
Validation Epoch Accuracy:16.74343727608572
save model /content/models/basic_pad/epoch_2.pt
Train Epoch Loss (Avg): 3.7492659091949463
Validation Epoch Accuracy:17.470199045252784
save model /content/models/basic_pad/epoch_3.pt
Train Epoch Loss (Avg): 3.7063188552856445
Validation Epoch Accuracy:17.931753254608562
save model /content/models/basic_pad/epoch_4.pt
Train Epoch Loss (Avg): 3.6707804203033447
Validation Epoch Accuracy:18.117863822897185
save model /content/models/basic_pad/epoch_5.pt
Train Epoch Loss (Avg): 3.6415538787841797
Validation Epoch Accuracy:18.14391930245759
save model /content/models/basic_pad/epoch_6.pt
Train Epoch Loss (Avg): 3.618687868118286
Validation